In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [2]:
#Download and parse the html
revenue_url = 'https://en.wikipedia.org/wiki/List_of_largest_manufacturing_companies_by_revenue'

In [3]:
#Download the HTML from start_url
downloaded_html = requests.get(revenue_url)

In [4]:
#Parse the html with BeautifulSoup and create a soup object
soup = BeautifulSoup(downloaded_html.text)

In [5]:
#soup = BeautifulSoup(downloaded_html.text, 'html.parser').encode("utf-8")
soup = BeautifulSoup(downloaded_html.text, 'html.parser')
#print(soup)
#Save a local copy 
#soup = bs4.BeautifulSoup(downloaded_html.text, 'html.parser')
with open('downloaded.html', 'w', encoding='utf-8') as file:
#with open ('downloaded.html', 'w') as file:
        file.write(soup.prettify())

In [9]:
manufacturing_table = soup.select('#mw-content-text > div.mw-parser-output > table tbody')[0]
print(manufacturing_table)

<tbody><tr>
<th scope="col">No.
</th>
<th scope="col">Company
</th>
<th scope="col">Industry
</th>
<th scope="col">Revenue (by US$ billion)
</th>
<th scope="col">Headquarters
</th></tr>
<tr>
<td>1</td>
<td><a href="/wiki/Volkswagen_Group" title="Volkswagen Group">Volkswagen Group</a></td>
<td>Automotive</td>
<td>282.7</td>
<td><span class="datasortkey" data-sort-value="Germany"><span class="flagicon"><img alt="" class="thumbborder" data-file-height="600" data-file-width="1000" decoding="async" height="14" src="//upload.wikimedia.org/wikipedia/en/thumb/b/ba/Flag_of_Germany.svg/23px-Flag_of_Germany.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/b/ba/Flag_of_Germany.svg/35px-Flag_of_Germany.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/b/ba/Flag_of_Germany.svg/46px-Flag_of_Germany.svg.png 2x" width="23"/> </span><a href="/wiki/Germany" title="Germany">Germany</a></span>
</td></tr>
<tr>
<td>2</td>
<td><a href="/wiki/Toyota_Group" title="Toyota Group">Toyota Group</a><

In [23]:
manufacturing_headers_columns = manufacturing_table.select('tr th')
manufacturing_headers = []
for element in manufacturing_headers_columns:
    header_names = element.text.replace('\n','')
    manufacturing_headers.append(header_names)
print(manufacturing_headers)

['No.', 'Company', 'Industry', 'Revenue (by US$ billion)', 'Headquarters']


In [58]:
import re
regex = re.compile('_\[\w\]')
manufacturing_rows = manufacturing_table.select('tr')

manufacturing_data = []
for index, element in enumerate(manufacturing_rows):
    if index > 0:
        row_list = []
        values = element.select('td')
        
        for value in values:
            label = value.get_text(strip=True)
            label = label.replace('\n', '')
            label = regex.sub('', label)
            row_list.append(label)
        
        manufacturing_data.append(row_list)
        
print(manufacturing_data)

[['1', 'Volkswagen Group', 'Automotive', '282.7', 'Germany'], ['2', 'Toyota Group', 'Engineering, various', '265.172*', 'Japan'], ['3', 'Apple', 'Electronics, telecommunications equipment', '260.174', 'United States'], ['4', 'Samsung Electronics', 'Electronics, various', '197.705', 'South Korea'], ['5', 'Daimler', 'Automotive', '193.346', 'Germany'], ['6', 'Foxconn', 'Electronics', '172.869', 'Taiwan'], ['7', 'Ford', 'Automotive', '155.900', 'United States'], ['8', 'Cardinal Health', 'Pharmaceuticals', '145.534', 'United States'], ['9', 'Honda', 'Automotive', '137.332', 'Japan'], ['10', 'General Motors', 'Automotive', '137.237', 'United States'], ['11', 'Mitsubishi', 'Engineering, various', '135.940', 'Japan'], ['12', 'Huawei', 'Telecommunications equipment, Electronics', '124.316', 'China'], ['13', 'China Railway and Engineering Group', 'Engineering', '123.324', 'China'], ['14', 'SAIC Motor', 'Automotive', '122.071', 'China'], ['15', 'BMW', 'Automotive', '116.638', 'Germany'], ['16', 

In [59]:
df = pd.DataFrame(manufacturing_data, columns=manufacturing_headers)
print(df)

    No.                       Company  \
0     1              Volkswagen Group   
1     2                  Toyota Group   
2     3                         Apple   
3     4           Samsung Electronics   
4     5                       Daimler   
..  ...                           ...   
95   96            Oracle Corporation   
96   97                   Tata Motors   
97   98                     Honeywell   
98   99                       Fujitsu   
99  100  China South Industries Group   

                                     Industry Revenue (by US$ billion)  \
0                                  Automotive                    282.7   
1                        Engineering, various                 265.172*   
2   Electronics, telecommunications equipment                  260.174   
3                        Electronics, various                  197.705   
4                                  Automotive                  193.346   
..                                        ...                  

In [60]:
df.to_csv('manufacturing_companies_revenue.csv')